In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from func import show_outliers

import torch
import torch.nn
import torch.nn.functional as F
import torch.optim as optim
from torchmetrics.classification import F1Score, BinaryF1Score, Accuracy
from torchinfo import summary
from torch.utils.data import DataLoader

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from get_dataset import *
from get_model import *
from get_train_model import *
from func import Torch_proccesing

In [36]:
rawDF= pd.read_csv('heart.csv')
rawDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246022 entries, 0 to 246021
Data columns (total 40 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   State                      246022 non-null  object 
 1   Sex                        246022 non-null  object 
 2   GeneralHealth              246022 non-null  object 
 3   PhysicalHealthDays         246022 non-null  float64
 4   MentalHealthDays           246022 non-null  float64
 5   LastCheckupTime            246022 non-null  object 
 6   PhysicalActivities         246022 non-null  object 
 7   SleepHours                 246022 non-null  float64
 8   RemovedTeeth               246022 non-null  object 
 9   HadHeartAttack             246022 non-null  object 
 10  HadAngina                  246022 non-null  object 
 11  HadStroke                  246022 non-null  object 
 12  HadAsthma                  246022 non-null  object 
 13  HadSkinCancer              24

In [37]:
rawDF= rawDF[1:]

In [38]:
for col in rawDF.columns:
    print(rawDF[col].value_counts())

State
Washington              15000
Maryland                 9165
Minnesota                9161
Ohio                     8995
New York                 8923
Texas                    7408
Florida                  7315
Kansas                   6145
Wisconsin                6126
Maine                    6013
Iowa                     5672
Hawaii                   5596
Virginia                 5565
Indiana                  5502
South Carolina           5471
Massachusetts            5465
Arizona                  5462
Utah                     5373
Michigan                 5370
Colorado                 5159
Nebraska                 5107
California               5096
Connecticut              5053
Georgia                  4978
Vermont                  4845
South Dakota             4405
Montana                  4264
Missouri                 4195
New Jersey               3967
New Hampshire            3756
Puerto Rico              3589
Idaho                    3468
Alaska                   3205
Rhod

In [39]:
# 이진분류 컬럼
binary_list= ['PhysicalActivities', 'HadHeartAttack', 'HadAngina', 'HadStroke', 'HadAsthma', 'HadSkinCancer', 'HadCOPD', 'HadDepressiveDisorder', 'HadKidneyDisease', 'HadArthritis', 'DeafOrHardOfHearing' , 'BlindOrVisionDifficulty',
'DifficultyConcentrating', 'DifficultyWalking', 'DifficultyDressingBathing', 'DifficultyErrands', 'ChestScan', 'HighRiskLastYear',
'CovidPos']
for col in binary_list:
    rawDF[col].replace({'Yes':1, 'No':0}, inplace=True)

In [40]:
rawDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246021 entries, 1 to 246021
Data columns (total 40 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   State                      246021 non-null  object 
 1   Sex                        246021 non-null  object 
 2   GeneralHealth              246021 non-null  object 
 3   PhysicalHealthDays         246021 non-null  float64
 4   MentalHealthDays           246021 non-null  float64
 5   LastCheckupTime            246021 non-null  object 
 6   PhysicalActivities         246021 non-null  int64  
 7   SleepHours                 246021 non-null  float64
 8   RemovedTeeth               246021 non-null  object 
 9   HadHeartAttack             246021 non-null  int64  
 10  HadAngina                  246021 non-null  int64  
 11  HadStroke                  246021 non-null  int64  
 12  HadAsthma                  246021 non-null  int64  
 13  HadSkinCancer              24

In [41]:
rawDF=rawDF[1:]

In [42]:
# 이진이 아닌 범주들-> 라벨 인코딩
for col in rawDF.columns:
    if rawDF[col].dtype== 'object':
        rawDF= pd.get_dummies(rawDF, columns=[col], drop_first=True,
                              sparse=False)

In [43]:
rawDF= rawDF.astype(float)

In [44]:
rawDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246020 entries, 2 to 246021
Columns: 122 entries, PhysicalHealthDays to CovidPos_Tested positive using home test without a health professional
dtypes: float64(122)
memory usage: 229.0 MB


In [45]:
rawDF.corr()

,PhysicalHealthDays,MentalHealthDays,PhysicalActivities,SleepHours,HadHeartAttack,HadAngina,HadStroke,HadAsthma,HadSkinCancer,HadCOPD,...,AgeCategory_Age 80 or older,AlcoholDrinkers_Yes,HIVTesting_Yes,FluVaxLast12_Yes,PneumoVaxEver_Yes,"TetanusLast10Tdap_Yes, received Tdap","TetanusLast10Tdap_Yes, received tetanus shot but not sure what type","TetanusLast10Tdap_Yes, received tetanus shot, but not Tdap",CovidPos_1,CovidPos_Tested positive using home test without a health professional
PhysicalHealthDays,1.000000,0.306799,-0.240842,-0.056064,0.133419,0.149130,0.128731,0.121698,0.030154,0.217158,...,0.029669,-0.129373,0.055651,0.016743,0.101222,-0.034189,0.017669,0.010678,0.025102,-0.004670
MentalHealthDays,0.306799,1.000000,-0.110529,-0.130100,0.025891,0.024085,0.042086,0.123013,-0.039538,0.099231,...,-0.077111,-0.025996,0.124786,-0.057819,-0.025417,0.016802,-0.010713,-0.001723,0.050046,-0.008780
PhysicalActivities,-0.240842,-0.110529,1.000000,0.003187,-0.083187,-0.078760,-0.079299,-0.043615,0.008118,-0.138524,...,-0.080240,0.157841,0.018496,0.030816,-0.051411,0.086192,0.007853,-0.004800,0.007009,0.032850
SleepHours,-0.056064,-0.130100,0.003187,1.000000,0.003632,0.011538,0.008333,-0.045030,0.044072,-0.018837,...,0.090811,-0.003537,-0.074168,0.066834,0.060529,-0.006733,0.004669,-0.002680,-0.048725,0.003218
HadHeartAttack,0.133419,0.025891,-0.083187,0.003632,1.000000,0.445903,0.177136,0.023755,0.049407,0.133222,...,0.100296,-0.074183,-0.014564,0.045237,0.119958,-0.040362,0.021736,0.009777,-0.016445,-0.022104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"TetanusLast10Tdap_Yes, received Tdap",-0.034189,0.016802,0.086192,-0.006733,-0.040362,-0.029032,-0.031132,0.040642,0.001627,-0.038644,...,-0.076719,0.061030,0.096915,0.119350,0.075755,1.000000,-0.415263,-0.187458,0.039616,0.023323
"TetanusLast10Tdap_Yes, received tetanus shot but not sure what type",0.017669,-0.010713,0.007853,0.004669,0.021736,0.021400,0.011717,-0.002766,0.014041,0.020303,...,-0.001381,0.017599,0.009277,0.006702,0.042183,-0.415263,1.000000,-0.194635,-0.002753,0.002575
"TetanusLast10Tdap_Yes, received tetanus shot, but not Tdap",0.010678,-0.001723,-0.004800,-0.002680,0.009777,0.012070,0.007762,-0.001858,0.014634,0.009721,...,0.006393,-0.004413,-0.005211,-0.014622,0.002227,-0.187458,-0.194635,1.000000,-0.002040,-0.009187
CovidPos_1,0.025102,0.050046,0.007009,-0.048725,-0.016445,-0.013979,-0.016789,0.044537,-0.034300,-0.007023,...,-0.079234,0.031247,0.075484,-0.070370,-0.073318,0.039616,-0.002753,-0.002040,1.000000,-0.118893


#### 심장병 예측 <hr>

In [46]:
rawDF= rawDF.reset_index(drop=True)
hTS=Torch_proccesing(rawDF)
hTS.feature=hTS.data.drop(columns='HadHeartAttack')
hTS.target= hTS.data['HadHeartAttack']

In [47]:
hTS.split(val=True, random_state=123,classification=True)

train: (138386, 121),(138386,)
test: (61505, 121),(61505,)
val: (46129, 121),(46129,)
train: (184515, 121),(184515,)
test: (61505, 121),(61505,)


In [64]:
torch.FloatTensor(hTS.X_train.iloc[:32].values)

tensor([[ 0.,  0.,  1.,  ...,  0.,  0.,  0.],
        [ 2.,  1.,  1.,  ...,  0.,  0.,  0.],
        [30.,  0.,  1.,  ...,  0.,  1.,  0.],
        ...,
        [ 0.,  0.,  0.,  ...,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  ...,  0.,  0.,  0.],
        [10., 15.,  1.,  ...,  0.,  0.,  0.]])

In [58]:
X_train=pd.DataFrame(hTS.X_train, columns=hTS.feature.columns)
X_test=pd.DataFrame(hTS.X_test, columns=hTS.feature.columns)
X_val=pd.DataFrame(hTS.X_val, columns=hTS.feature.columns)

In [79]:
class C_Dataset(Dataset):
    def __init__(self, featureDF, targetDF) -> None:
        super().__init__()
        self.featureDF= featureDF
        self.targetDF= targetDF
        self.n_rows= featureDF.shape[0]
        self.n_features= featureDF.shape[1]
    
    def __len__(self):
        return self.n_rows
    
    def __getitem__(self, index):
        featureTS= torch.FloatTensor(self.featureDF.iloc[index].values)
        targetTS= torch.FloatTensor(self.targetDF.iloc[index].values)
        return featureTS, targetTS

In [86]:
trainDS= C_Dataset(X_train, hTS.Y_train.to_frame())
testDS= C_Dataset(X_test, hTS.Y_test.to_frame())
valDS= C_Dataset(X_val, hTS.Y_val.to_frame())
trainDL= DataLoader(trainDS, batch_size=32)
testDL= DataLoader(testDS, batch_size=32)
valDL= DataLoader(valDS, batch_size=32)

In [88]:
X_train.iloc[:32].values

array([[ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 2.,  1.,  1., ...,  0.,  0.,  0.],
       [30.,  0.,  1., ...,  0.,  1.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [10., 15.,  1., ...,  0.,  0.,  0.]])

In [89]:
for f, t in trainDL:
    print(f, t)
    print(f.shape, t.shape)
    break

tensor([[ 0.,  0.,  1.,  ...,  0.,  0.,  0.],
        [ 2.,  1.,  1.,  ...,  0.,  0.,  0.],
        [30.,  0.,  1.,  ...,  0.,  1.,  0.],
        ...,
        [ 0.,  0.,  0.,  ...,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  ...,  0.,  0.,  0.],
        [10., 15.,  1.,  ...,  0.,  0.,  0.]]) tensor([[0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]])
torch.Size([32, 121]) torch.Size([32, 1])


In [100]:
class make_model(nn.Module):
    '''
    커스텀 모델을 만드는 함수
    model_type= 'reg'|'binary'|'mclf'
    은닉층 수= 리스트 수-1
    '''
    def __init__(self, in_in, out_out, hidden: list) -> None:
        super().__init__()


        self.in_layer= nn.Linear(in_in, hidden[0])
        self.h1_layer=nn.Linear(hidden[0], hidden[1])
        self.h2_layer=nn.Linear(hidden[1], hidden[2])
        self.h3_layer=nn.Linear(hidden[2], hidden[3])
        self.out_layer= nn.Linear(hidden[-1], out_out)
    
    def forward(self, input_data):
        
        y= self.in_layer(input_data)
        y= F.relu(y)

        
        y=self.h1_layer(y)
        y=F.relu(y)
        
        y=self.h2_layer(y)
        y=F.relu(y)

        y=self.h3_layer(y)
        y=F.relu(y)

        y=self.out_layer(y)
        y=F.sigmoid(y)
        
        return y

In [102]:
hidden=[150,120,100,80]

model= make_model(in_in=121, out_out=1, hidden=hidden)

In [103]:
# 설정값
EPOCH=100
DEVICE= 'cuda' if torch.cuda.is_available() else 'cpu'
LR= 0.01
torch.manual_seed(1)
# 최적화 인스턴스
optimizer= optim.Adam(model.parameters(), lr=LR)

# 저장
import os
# 경로
SAVE_PATH= '../models/heart/'
# 이름
SAVE_FILE='model_train_all.pth'

if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)       #하위 폴더까지 생성

In [104]:
model=model
trainDL=trainDL; testDL=valDL;  optimizer=optimizer;  epoch=100
break_param='score'; mtype='binary'; SAVE_FILE=SAVE_FILE; SAVE_PATH=SAVE_PATH; save_type='all'
numcls=5; LIMIT=15; optim_type='score'

In [107]:
scheduler= lr_scheduler.ReduceLROnPlateau(optimizer, patience=LIMIT, mode='max')
EPOCH=100
# 손실, 평가값 저장
LOSS_HISTORY, SCORE_HISTORY= [[],[]], [[],[]]
for ep in range(EPOCH):
    print(f'{ep+1}/{EPOCH}')
    model.train()
    loss_total, score_total= 0,0
    loss_val_total, score_val_total=0,0

    for train_feature, train_target in trainDL:
        # 학습
        pre_y=model(train_feature)

        # 손실
        # if type=='reg':
        #     Lossfunc=MeanSquaredError()
        #     Scorefunc=R2Score()
        # elif type=='binary':
        #     Lossfunc= nn.BCELoss()
        #     Scorefunc=F1Score(task='binary')
        # elif type=='muticlass':
        #     Lossfunc=nn.CrossEntropyLoss()
        #     Scorefunc=F1Score(task='multiclass', num_classes=numcls)
        Lossfunc= nn.BCELoss()
        Scorefunc=F1Score(task='binary')

        
        loss= Lossfunc(pre_y, train_target.reshape(-1).long() if type=='muticlass' else train_target)
        loss_total+=loss.item()

        # 평가
        score= Scorefunc(pre_y, train_target.reshape(-1) if type=='muticlass' else train_target if type=='reg' else train_target)
        score_total+=score.item()
        # 최적화
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 검증

    model.eval()
    with torch.no_grad():
        for val_feature, val_target in testDL:
            # 학습
            pre_val= model(val_feature)

            # 손실
            loss= Lossfunc(pre_val, val_target.reshape(-1).long() if type=='muticlass' else val_target if type=='reg' else val_target)
            loss_val_total+=loss.item()

            # 평가
            score= Scorefunc(pre_val, val_target.reshape(-1) if type=='muticlass' else val_target)
            score_val_total+=score.item()

    
    # 저장
    LOSS_HISTORY[0].append(loss_total/len(trainDL))
    SCORE_HISTORY[0].append(score_total/len(trainDL))
    print(f'Train\n Loss: {loss_total/len(trainDL)}\n Score: {score_total/len(trainDL)}')

    LOSS_HISTORY[1].append(loss_val_total/len(testDL))
    SCORE_HISTORY[1].append(score_val_total/len(testDL))
    print(f'Val\n Loss: {loss_val_total/len(testDL)}\n Score: {score_val_total/len(testDL)}')

    # 성능이 좋은 학습 가중치 저장
    if save_type:
        if save_type=='all':
            save_type= model
        elif save_type=='param':
            save_type=model.state_dict()
        if len(SCORE_HISTORY[1]) == 1: 
        #첫번째는 무조건 저장
            torch.save(save_type, SAVE_PATH+SAVE_FILE)  
            
        else:
            if SCORE_HISTORY[1][-1]> max(SCORE_HISTORY[1][:-1]): # 자신을 제외한 최대점수값과 비교
                torch.save(save_type, SAVE_PATH+SAVE_FILE) 
                    
    else: pass

    
    # 학습 진행 모니터링 (검증 데이터 개선이 되지 않았을때 누적 ->  평가, 손실 중 지표 하나 선택)
    # 최적화 스케쥴러 인스턴스 업데이트
    # scheduler.step(score_val_total/len(testDL))
    # print(f'scheduler.num_bad_epochs: {scheduler.num_bad_epochs}', end=' ') #보여주기용
    # print(f'scheduler.patience: {scheduler.patience}')
    # 손실 감소 (또는 성능 개선)이 안되는 경우 조기종료
    # if scheduler.num_bad_epochs== scheduler.patience:
    #     print(f'{scheduler.patience} EPOCH 성능 개선이 없어서 조기종료함')
    #     break



1/100
Train
 Loss: 0.15071672723784096
 Score: 0.17666001082272842
Val
 Loss: 0.14735875452962935
 Score: 0.0888828357859888
2/100
Train
 Loss: 0.14853147824314406
 Score: 0.14943474935032125
Val
 Loss: 0.1493467137362193
 Score: 0.0
3/100
Train
 Loss: 0.1495897158127209
 Score: 0.14844427795349796
Val
 Loss: 0.14776125033327925
 Score: 0.056574863695569244
4/100
Train
 Loss: 0.14807151620395292
 Score: 0.1651967567294314
Val
 Loss: 0.14777831160224428
 Score: 0.020399907797616332
5/100
Train
 Loss: 0.14854774467155496
 Score: 0.1668071532985336
Val
 Loss: 0.14743745284369436
 Score: 0.0
6/100
Train
 Loss: 0.1482110448105405
 Score: 0.17072205830223952
Val
 Loss: 0.1482431987163594
 Score: 0.04785075906700301
7/100
Train
 Loss: 0.14830228042836766
 Score: 0.1620131080932379
Val
 Loss: 0.14753152808915931
 Score: 0.0
8/100
Train
 Loss: 0.14796702473763346
 Score: 0.1512772474242224
Val
 Loss: 0.14695884923782618
 Score: 0.23311043185557803
9/100
Train
 Loss: 0.14805379504499722
 Score: 

KeyboardInterrupt: 

In [97]:
model_training(model, trainDL=trainDL, testDL=valDL, optimizer=optimizer, epoch=100,
               break_param='score', type='binary', SAVE_FILE=SAVE_FILE, SAVE_PATH=SAVE_PATH, save_type='all',
               numcls=None, LIMIT=15, optim_type='score')

1/100


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x120 and 80x1)